In [331]:
import pandas as pd
import os

In [332]:
asset_a_path = '/home/nick/Documents/money/temp/drive-download-20231028T180526Z-001/lagged_Correlation/SPY_15Min_3days.csv'
asset_b_path = '/home/nick/Documents/money/temp/drive-download-20231028T180526Z-001/lagged_Correlation/SPY_15Min_3days.csv'

In [333]:
asset_a_df = pd.read_csv(asset_a_path)
asset_b_df = pd.read_csv(asset_b_path)

In [334]:
high_thres = 0
low_thres = 0

In [335]:
lookback = 15
lookahead = 15

In [336]:
# convert Date column to datetime
asset_a_df['Time'] = pd.to_datetime(asset_a_df['Time'])

# sort by date
asset_a_df.sort_values(by='Time', inplace=True)
asset_a_df.reset_index(drop=True, inplace=True)
asset_a_change = asset_a_df[['Time', 'Open']]

asset_b_df['Time'] = pd.to_datetime(asset_b_df['Time'])
asset_b_df.sort_values(by='Time', inplace=True)
asset_b_df.reset_index(drop=True, inplace=True)
asset_b_change = asset_b_df[['Time', 'Open']]

In [337]:
asset_a_change

,Time,Open
0,2023-11-09 09:30:00,438.4300
1,2023-11-09 09:45:00,437.4200
2,2023-11-09 10:00:00,437.8800
3,2023-11-09 10:15:00,436.8968
4,2023-11-09 10:30:00,436.5100
...,...,...
73,2023-11-13 14:45:00,440.9850
74,2023-11-13 15:00:00,440.5900
75,2023-11-13 15:15:00,440.7300
76,2023-11-13 15:30:00,440.7500


In [338]:
# Calculate the difference in Time between each row
time_delta = asset_a_change['Time'].diff()

# convert into minutes and count the frequency of each time difference
time_delta = time_delta.dt.total_seconds() / 60
time_delta = time_delta.value_counts().reset_index()
time_delta.columns = ['Time_Delta', 'Frequency']

# get the most frequent time difference
time_delta = time_delta.sort_values('Frequency', ascending=False)
time_delta = time_delta.reset_index(drop=True)
time_delta = time_delta['Time_Delta'][0]
time_delta

15.0

In [339]:
lookback_shift = int(lookback / time_delta)

# make a mask for if each row's Time minus the Time for a row lookback_shift rows ago is equal to the lookback
lookback_mask = asset_a_change['Time'].diff(lookback_shift) == pd.Timedelta(minutes=lookback)
lookback_mask

0     False
1      True
2      True
3      True
4      True
      ...  
73     True
74     True
75     True
76     True
77     True
Name: Time, Length: 78, dtype: bool

In [340]:
asset_a_change.head(10)

,Time,Open
0,2023-11-09 09:30:00,438.4300
1,2023-11-09 09:45:00,437.4200
2,2023-11-09 10:00:00,437.8800
3,2023-11-09 10:15:00,436.8968
4,2023-11-09 10:30:00,436.5100
5,2023-11-09 10:45:00,436.6300
6,2023-11-09 11:00:00,436.9700
7,2023-11-09 11:15:00,437.5800
8,2023-11-09 11:30:00,437.4300
9,2023-11-09 11:45:00,437.5250


In [341]:
# for the rows in lookback_mask, calculate the difference between the Last price and the Open price lookback_shift rows ago
lookback_pct_change = asset_a_change['Open'].pct_change(lookback_shift) * 100
lookback_change = asset_a_change['Open'].diff(lookback_shift)

# # set the lookback_pct_change for the rows not in lookback_mask to NaN
lookback_pct_change.loc[~lookback_mask] = pd.NaT
lookback_change.loc[~lookback_mask] = pd.NaT

asset_a_change.loc[lookback_mask, 'Chg%'] = lookback_pct_change
asset_a_change.loc[lookback_mask, 'Chg'] = lookback_change

/tmp/ipykernel_123565/365138428.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change.loc[lookback_mask, 'Chg%'] = lookback_pct_change
/tmp/ipykernel_123565/365138428.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change.loc[lookback_mask, 'Chg'] = lookback_change


In [342]:
# split datetime into date and time
asset_a_change['Date'] = asset_a_change['Time'].dt.date
asset_a_change['Time'] = asset_a_change['Time'].dt.time

# convert both to datetime objects
asset_a_change['Date'] = pd.to_datetime(asset_a_change['Date'])
asset_a_change['Time'] = pd.to_datetime(asset_a_change['Time'], format='%H:%M:%S')

/tmp/ipykernel_123565/2151251943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change['Date'] = asset_a_change['Time'].dt.date
/tmp/ipykernel_123565/2151251943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change['Time'] = asset_a_change['Time'].dt.time
/tmp/ipykernel_123565/2151251943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [343]:
asset_a_change.head(10)

,Time,Open,Chg%,Chg,Date
0,1900-01-01 09:30:00,438.4300,NaN,NaN,2023-11-09
1,1900-01-01 09:45:00,437.4200,-0.230367,-1.01,2023-11-09
2,1900-01-01 10:00:00,437.8800,0.105162,0.46,2023-11-09
3,1900-01-01 10:15:00,436.8968,-0.224536,-0.9832,2023-11-09
4,1900-01-01 10:30:00,436.5100,-0.088533,-0.3868,2023-11-09
5,1900-01-01 10:45:00,436.6300,0.027491,0.12,2023-11-09
6,1900-01-01 11:00:00,436.9700,0.077869,0.34,2023-11-09
7,1900-01-01 11:15:00,437.5800,0.139598,0.61,2023-11-09
8,1900-01-01 11:30:00,437.4300,-0.034279,-0.15,2023-11-09
9,1900-01-01 11:45:00,437.5250,0.021718,0.095,2023-11-09


In [344]:
asset_a_df.head()

,Time,Open,High,Low,Last,Change,%Chg,Volume
0,2023-11-09 09:30:00,438.4300,438.4700,437.21,437.4200,0.1700,0.04%,3072338
1,2023-11-09 09:45:00,437.4200,437.8800,436.98,437.8800,0.4600,0.11%,1745455
2,2023-11-09 10:00:00,437.8800,438.1593,436.43,436.8950,-0.9850,-0.22%,3002670
3,2023-11-09 10:15:00,436.8968,436.9700,436.36,436.4876,-0.4074,-0.09%,1497095
4,2023-11-09 10:30:00,436.5100,436.7275,436.08,436.6500,0.1624,0.04%,1842614


In [345]:
# remove % in the %Chg column values of asset_a_df
asset_a_df['%Chg'] = asset_a_df['%Chg'].str.replace('%', '')

In [346]:
# asset_a_change['Chg%'] = asset_a_df['%Chg'].astype(float)

In [347]:
asset_a_change_pivot = asset_a_change.pivot(index='Date', columns='Time', values='Chg%')
# drop columns that are all NaN
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')

# drop rows that are all NaN
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=0, how='all')
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')

asset_a_change_orig = asset_a_change.copy()
asset_a_change_pivot

Time,1900-01-01 09:45:00,1900-01-01 10:00:00,1900-01-01 10:15:00,1900-01-01 10:30:00,1900-01-01 10:45:00,1900-01-01 11:00:00,1900-01-01 11:15:00,1900-01-01 11:30:00,1900-01-01 11:45:00,1900-01-01 12:00:00,...,1900-01-01 13:30:00,1900-01-01 13:45:00,1900-01-01 14:00:00,1900-01-01 14:15:00,1900-01-01 14:30:00,1900-01-01 14:45:00,1900-01-01 15:00:00,1900-01-01 15:15:00,1900-01-01 15:30:00,1900-01-01 15:45:00
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-09,-0.230367,0.105162,-0.224536,-0.088533,0.027491,0.077869,0.139598,-0.034279,0.021718,0.065139,...,-0.057254,-0.162695,-0.05738,-0.212429,0.063289,-0.209296,-0.008067,0.079521,-0.094429,-0.059938
2023-11-10,0.036699,-0.077957,-0.131941,-0.098799,-0.01035,0.144911,0.314668,-0.025186,0.026337,0.003434,...,0.138005,0.145788,-0.023883,0.059154,0.009095,-0.018189,-0.020466,0.036983,0.123322,0.046552
2023-11-13,-0.134326,0.050155,0.070638,0.0,0.059203,0.065995,0.088693,0.088614,0.065834,0.04991,...,-0.063571,0.140855,-0.060891,0.033687,0.059001,0.012474,-0.089572,0.031776,0.004538,-0.05899


In [348]:
asset_a_change_pivot = asset_a_change_pivot.applymap(lambda x: 1 if x >= high_thres else -1 if x <= -low_thres else pd.NaT)

# drop columns that are all NaN
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=0, how='all')
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')

asset_a_change_pivot

Time,1900-01-01 09:45:00,1900-01-01 10:00:00,1900-01-01 10:15:00,1900-01-01 10:30:00,1900-01-01 10:45:00,1900-01-01 11:00:00,1900-01-01 11:15:00,1900-01-01 11:30:00,1900-01-01 11:45:00,1900-01-01 12:00:00,...,1900-01-01 13:30:00,1900-01-01 13:45:00,1900-01-01 14:00:00,1900-01-01 14:15:00,1900-01-01 14:30:00,1900-01-01 14:45:00,1900-01-01 15:00:00,1900-01-01 15:15:00,1900-01-01 15:30:00,1900-01-01 15:45:00
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-09,-1,1,-1,-1,1,1,1,-1,1,1,...,-1,-1,-1,-1,1,-1,-1,1,-1,-1
2023-11-10,1,-1,-1,-1,-1,1,1,-1,1,1,...,1,1,-1,1,1,-1,-1,1,1,1
2023-11-13,-1,1,1,1,1,1,1,1,1,1,...,-1,1,-1,1,1,1,-1,1,1,-1


In [349]:
asset_b_change

,Time,Open
0,2023-11-09 09:30:00,438.4300
1,2023-11-09 09:45:00,437.4200
2,2023-11-09 10:00:00,437.8800
3,2023-11-09 10:15:00,436.8968
4,2023-11-09 10:30:00,436.5100
...,...,...
73,2023-11-13 14:45:00,440.9850
74,2023-11-13 15:00:00,440.5900
75,2023-11-13 15:15:00,440.7300
76,2023-11-13 15:30:00,440.7500


In [350]:
# Calculate the difference in Time between each row
time_delta_b = asset_b_change['Time'].diff()

# convert into minutes and count the frequency of each time difference
time_delta_b = time_delta_b.dt.total_seconds() / 60
time_delta_b = time_delta_b.value_counts().reset_index()
time_delta_b.columns = ['Time_Delta', 'Frequency']

# get the most frequent time difference
time_delta_b = time_delta_b.sort_values('Frequency', ascending=False)
time_delta_b = time_delta_b.reset_index(drop=True)
time_delta_b = time_delta_b['Time_Delta'][0]
time_delta_b

15.0

In [351]:
lookahead_shift = int(lookahead / time_delta_b)

lookahead_mask = asset_b_change['Time'].diff(-lookahead_shift) == pd.Timedelta(minutes=-lookahead)

print(lookahead_mask)

0      True
1      True
2      True
3      True
4      True
      ...  
73     True
74     True
75     True
76     True
77    False
Name: Time, Length: 78, dtype: bool


In [352]:
lookahead_pct_change = asset_b_change['Open'].pct_change(lookahead_shift).shift(-lookahead_shift) * 100
lookagead_change = asset_b_change['Open'].diff(lookahead_shift).shift(-lookahead_shift)

lookahead_pct_change.loc[~lookahead_mask] = pd.NaT
lookagead_change.loc[~lookahead_mask] = pd.NaT

asset_b_change.loc[lookahead_mask, 'Chg%'] = lookahead_pct_change
asset_b_change.loc[lookahead_mask, 'Chg'] = lookagead_change

/tmp/ipykernel_123565/3598038920.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change.loc[lookahead_mask, 'Chg%'] = lookahead_pct_change
/tmp/ipykernel_123565/3598038920.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change.loc[lookahead_mask, 'Chg'] = lookagead_change


In [353]:
asset_b_change

,Time,Open,Chg%,Chg
0,2023-11-09 09:30:00,438.4300,-0.230367,-1.01
1,2023-11-09 09:45:00,437.4200,0.105162,0.46
2,2023-11-09 10:00:00,437.8800,-0.224536,-0.9832
3,2023-11-09 10:15:00,436.8968,-0.088533,-0.3868
4,2023-11-09 10:30:00,436.5100,0.027491,0.12
...,...,...,...,...
73,2023-11-13 14:45:00,440.9850,-0.089572,-0.395
74,2023-11-13 15:00:00,440.5900,0.031776,0.14
75,2023-11-13 15:15:00,440.7300,0.004538,0.02
76,2023-11-13 15:30:00,440.7500,-0.05899,-0.26


In [354]:
asset_b_change['Date'] = asset_b_change['Time'].dt.date
asset_b_change['Time'] = asset_b_change['Time'].dt.time

asset_b_change['Date'] = pd.to_datetime(asset_b_change['Date'])
asset_b_change['Time'] = pd.to_datetime(asset_b_change['Time'], format='%H:%M:%S')

/tmp/ipykernel_123565/2585082788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change['Date'] = asset_b_change['Time'].dt.date
/tmp/ipykernel_123565/2585082788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change['Time'] = asset_b_change['Time'].dt.time
/tmp/ipykernel_123565/2585082788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [355]:
asset_b_change_pivot = asset_b_change.pivot(index='Date', columns='Time', values='Chg%')
asset_b_vals_pivot = asset_b_change.pivot(index='Date', columns='Time', values='Chg')

asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=0, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')

asset_b_change_orig = asset_b_change_pivot.copy()

asset_b_change_pivot = asset_b_change_pivot.applymap(lambda x: 1 if x > 0 else -1 if x <= 0 else pd.NaT)

asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=0, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')

In [356]:
asset_b_change_orig

Time,1900-01-01 09:30:00,1900-01-01 09:45:00,1900-01-01 10:00:00,1900-01-01 10:15:00,1900-01-01 10:30:00,1900-01-01 10:45:00,1900-01-01 11:00:00,1900-01-01 11:15:00,1900-01-01 11:30:00,1900-01-01 11:45:00,...,1900-01-01 13:15:00,1900-01-01 13:30:00,1900-01-01 13:45:00,1900-01-01 14:00:00,1900-01-01 14:15:00,1900-01-01 14:30:00,1900-01-01 14:45:00,1900-01-01 15:00:00,1900-01-01 15:15:00,1900-01-01 15:30:00
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-09,-0.230367,0.105162,-0.224536,-0.088533,0.027491,0.077869,0.139598,-0.034279,0.021718,0.065139,...,-0.057254,-0.162695,-0.05738,-0.212429,0.063289,-0.209296,-0.008067,0.079521,-0.094429,-0.059938
2023-11-10,0.036699,-0.077957,-0.131941,-0.098799,-0.01035,0.144911,0.314668,-0.025186,0.026337,0.003434,...,0.138005,0.145788,-0.023883,0.059154,0.009095,-0.018189,-0.020466,0.036983,0.123322,0.046552
2023-11-13,-0.134326,0.050155,0.070638,0.0,0.059203,0.065995,0.088693,0.088614,0.065834,0.04991,...,-0.063571,0.140855,-0.060891,0.033687,0.059001,0.012474,-0.089572,0.031776,0.004538,-0.05899


In [357]:
# For the two pivot tables, find the Date rows and Time columns that are in both tables
common_dates = asset_a_change_pivot.index.intersection(asset_b_change_pivot.index)
common_times = asset_a_change_pivot.columns.intersection(asset_b_change_pivot.columns)

# slice the pivot tables to only include the common dates and times
asset_a_change_pivot = asset_a_change_pivot.loc[common_dates, common_times]
asset_b_change_pivot = asset_b_change_pivot.loc[common_dates, common_times]

In [358]:
# set the rows and columns of asset_b_vals_pivot to match the rows and columns of asset_a_change_pivot
asset_b_vals_pivot = asset_b_vals_pivot.loc[common_dates, common_times]

In [359]:
asset_a_change_pivot

Time,1900-01-01 09:45:00,1900-01-01 10:00:00,1900-01-01 10:15:00,1900-01-01 10:30:00,1900-01-01 10:45:00,1900-01-01 11:00:00,1900-01-01 11:15:00,1900-01-01 11:30:00,1900-01-01 11:45:00,1900-01-01 12:00:00,...,1900-01-01 13:15:00,1900-01-01 13:30:00,1900-01-01 13:45:00,1900-01-01 14:00:00,1900-01-01 14:15:00,1900-01-01 14:30:00,1900-01-01 14:45:00,1900-01-01 15:00:00,1900-01-01 15:15:00,1900-01-01 15:30:00
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-09,-1,1,-1,-1,1,1,1,-1,1,1,...,-1,-1,-1,-1,-1,1,-1,-1,1,-1
2023-11-10,1,-1,-1,-1,-1,1,1,-1,1,1,...,-1,1,1,-1,1,1,-1,-1,1,1
2023-11-13,-1,1,1,1,1,1,1,1,1,1,...,-1,-1,1,-1,1,1,1,-1,1,1


In [360]:
asset_b_change_pivot

Time,1900-01-01 09:45:00,1900-01-01 10:00:00,1900-01-01 10:15:00,1900-01-01 10:30:00,1900-01-01 10:45:00,1900-01-01 11:00:00,1900-01-01 11:15:00,1900-01-01 11:30:00,1900-01-01 11:45:00,1900-01-01 12:00:00,...,1900-01-01 13:15:00,1900-01-01 13:30:00,1900-01-01 13:45:00,1900-01-01 14:00:00,1900-01-01 14:15:00,1900-01-01 14:30:00,1900-01-01 14:45:00,1900-01-01 15:00:00,1900-01-01 15:15:00,1900-01-01 15:30:00
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-09,1,-1,-1,1,1,1,-1,1,1,-1,...,-1,-1,-1,-1,1,-1,-1,1,-1,-1
2023-11-10,-1,-1,-1,-1,1,1,-1,1,1,1,...,1,1,-1,1,1,-1,-1,1,1,1
2023-11-13,1,1,-1,1,1,1,1,1,1,-1,...,-1,1,-1,1,1,1,-1,1,1,-1


In [361]:
probs_df = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])
dist_df = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])
trade_df = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])

In [362]:
time_diff = 0

In [363]:
# for each column in asset_a_change_pivot
for col in asset_a_change_pivot.columns:
    asset_a_col = asset_a_change_pivot[col]

    # shift datetime.time by time_diff
    b_col = col + pd.Timedelta(minutes=time_diff)

    if b_col not in asset_b_change_pivot.columns:
        continue

    asset_b_col = asset_b_change_pivot[b_col]

    # get the combinations in tuple pairs, and any pairs with NaN are dropped
    combo_col = pd.DataFrame(list(zip(asset_a_col, asset_b_col))).dropna()

    asset_a_prob = combo_col[0].value_counts(normalize=True)
    conditional_probs = combo_col.groupby(0)[1].value_counts(normalize=True)

    conditional = conditional_probs.unstack().fillna(0).to_dict()

    full_conditional = {}

    full_conditional_stats = {}

    trade_stats = {}

    rename = {-1: "down", 1: "up"}

    for i in [-1, 1]:
        for j in [-1, 1]:
            if i not in conditional or j not in conditional[i]:
                full_conditional[f"a_{rename[j]}_b_{rename[i]}"] = 0
                full_conditional_stats[f"a_{rename[j]}_b_{rename[i]}"] = [0,0,0]
                trade_stats[f"a_{rename[j]}_b_{rename[i]}"] = []

            else:
                full_conditional[f"a_{rename[j]}_b_{rename[i]}"] = conditional[i][j]

                # get the rows for each combination of a and b
                a_down_b_down = combo_col[(combo_col[0] == j) & (combo_col[1] == i)]
                
                #print(asset_b_change_orig.iloc[a_down_b_down.index][b_col])
                # get the count, mean, and std from asset_b_vals_pivot for the rows in a_down_b_down for the hour
                a_down_b_down_vals = asset_b_change_orig.iloc[a_down_b_down.index][b_col].astype(float).describe().fillna(0).to_dict()
                #print(a_down_b_down_vals)
                full_conditional_stats[f"a_{rename[j]}_b_{rename[i]}"] = [int(a_down_b_down_vals['count']), a_down_b_down_vals['mean'], a_down_b_down_vals['std']]

                trade_history = asset_b_vals_pivot.iloc[a_down_b_down.index][b_col].astype(float).to_dict()
                
                # convert the dictionary to a list of tuples, converting Timestamp to string of only the date
                trade_history = [(str(k.date()), v) for k, v in trade_history.items()]

                trade_stats[f"a_{rename[j]}_b_{rename[i]}"] = trade_history

    name = col.strftime('%H:%M:%S')
    
    # # add the percentages to the dataframe using the name as the index
    probs_df.loc[name] = full_conditional
    dist_df.loc[name] = full_conditional_stats
    trade_df.loc[name] = trade_stats

In [368]:
probs_df

,a_down_b_down,a_down_b_up,a_up_b_down,a_up_b_up
09:45:00,0.000000,1.000000,1.000000,0.000000
10:00:00,1.000000,0.000000,0.500000,0.500000
10:15:00,1.000000,0.000000,1.000000,0.000000
10:30:00,0.500000,0.500000,0.000000,1.000000
10:45:00,0.000000,1.000000,0.000000,1.000000
11:00:00,0.000000,0.000000,0.000000,1.000000
11:15:00,0.000000,0.000000,0.666667,0.333333
11:30:00,0.000000,1.000000,0.000000,1.000000
11:45:00,0.000000,0.000000,0.000000,1.000000
12:00:00,0.000000,0.000000,0.666667,0.333333


In [365]:
dist_df

,a_down_b_down,a_down_b_up,a_up_b_down,a_up_b_up
09:45:00,"[0, 0.0, 0.0]","[2, 0.07765855570156521, 0.03889586666649823]","[1, -0.07795661943411636, 0.0]","[0, 0.0, 0.0]"
10:00:00,"[1, -0.13194125745754626, 0.0]","[0, 0.0, 0.0]","[1, -0.22453640266739683, 0.0]","[1, 0.07063756095337936, 0.0]"
10:15:00,"[2, -0.09366648251545873, 0.007259142690785402]","[0, 0, 0]","[1, 0.0, 0.0]","[0, 0, 0]"
10:30:00,"[1, -0.010349705033407819, 0.0]","[1, 0.027490779134509147, 0.0]","[0, 0.0, 0.0]","[1, 0.05920258669762646, 0.0]"
10:45:00,"[0, 0, 0]","[1, 0.1449108683151179, 0.0]","[0, 0, 0]","[2, 0.07193185897110821, 0.008396574939309946]"
11:00:00,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[3, 0.18098604507232144, 0.11853651184851698]"
11:15:00,"[0, 0, 0]","[0, 0, 0]","[2, -0.029732739621962656, 0.006430013886385891]","[1, 0.08861421008383719, 0.0]"
11:30:00,"[0, 0, 0]","[2, 0.024027623407341103, 0.003266639371627081]","[0, 0, 0]","[1, 0.06583427922814966, 0.0]"
11:45:00,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[3, 0.03949464760935051, 0.032143952876227545]"
12:00:00,"[0, 0, 0]","[0, 0, 0]","[2, -0.08530554921744593, 0.01404368723343688]","[1, 0.11449962222680465, 0.0]"


In [366]:
trade_df

,a_down_b_down,a_down_b_up,a_up_b_down,a_up_b_up
09:45:00,[],"[(2023-11-09, 0.45999999999997954), (2023-11-1...","[(2023-11-10, -0.339999999999975)]",[]
10:00:00,"[(2023-11-10, -0.5749999999999886)]",[],"[(2023-11-09, -0.9832000000000107)]","[(2023-11-13, 0.3100000000000023)]"
10:15:00,"[(2023-11-09, -0.3867999999999938), (2023-11-1...",[],"[(2023-11-13, 0.0)]",[]
10:30:00,"[(2023-11-10, -0.045000000000015916)]","[(2023-11-09, 0.12000000000000455)]",[],"[(2023-11-13, 0.2599999999999909)]"
10:45:00,[],"[(2023-11-10, 0.6299999999999955)]",[],"[(2023-11-09, 0.34000000000003183), (2023-11-1..."
11:00:00,[],[],[],"[(2023-11-09, 0.6099999999999568), (2023-11-10..."
11:15:00,[],[],"[(2023-11-09, -0.14999999999997726), (2023-11-...","[(2023-11-13, 0.38999999999998636)]"
11:30:00,[],"[(2023-11-09, 0.09499999999997044), (2023-11-1...",[],"[(2023-11-13, 0.29000000000002046)]"
11:45:00,[],[],[],"[(2023-11-09, 0.285000000000025), (2023-11-10,..."
12:00:00,[],[],"[(2023-11-09, -0.3299999999999841), (2023-11-1...","[(2023-11-10, 0.5001000000000317)]"
